## Download data from API

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime

### Download API token from file

In [2]:
def get_api_token_from_file(filename):
    try:
        with open(filename, 'r') as file:
            api_token = file.read().strip()
        return api_token
    except FileNotFoundError:
        print(f"Plik {filename} nie został znaleziony.")
        return None
    except Exception as e:
        print(f"Wystąpił błąd podczas odczytu pliku {filename}: {str(e)}")
        return None
    
api_token_file = 'd:\\praca_inzynierska\\Pliki_projektu\\api_token.txt'
my_api_token = get_api_token_from_file(api_token_file)
if my_api_token:
    print(f"Pobrano API token z pliku")
else:
    print("Nie udało się pobrać API token.")

Pobrano API token z pliku


### Get player attribute data from API and create CSV file

In [22]:
headers = {
    'X-Auth-Token': my_api_token
}

def get_data_from_api(url):
    while True:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            wait_time = int(response.json().get('message').split()[-2])
            print(f"Limit żądań przekroczony. Czekanie {wait_time} sekund.")
            time.sleep(wait_time)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

def get_all_teams():
    url = 'https://api.football-data.org/v2/teams'
    data = get_data_from_api(url)
    if data:
        teams = data['teams']
        return pd.DataFrame(teams)
    return pd.DataFrame()

def get_players_for_team(team_id):
    url = f'https://api.football-data.org/v2/teams/{team_id}'
    data = get_data_from_api(url)
    if data and 'squad' in data:
        players = data['squad']
        return pd.DataFrame(players)
    return pd.DataFrame()

def get_player_attributes_from_2022(player_id):
    url = f'https://api.football-data.org/v2/players/{player_id}'
    data = get_data_from_api(url)
    if data:
        last_update = datetime.strptime(data['lastUpdated'], '%Y-%m-%dT%H:%M:%SZ')
        if last_update.year >= 2022:
            return data
    return None

def collect_data_from_2022():
    teams = get_all_teams()
    all_players = []
    
    for _, team in teams.iterrows():
        team_id = team['id']
        players = get_players_for_team(team_id)
        if not players.empty:
            players['team_id'] = team_id
            players['team_name'] = team['name']
            players['league_id'] = team['area']['id']
            players['league_name'] = team['area']['name']
            all_players.append(players)
        time.sleep(1)
    
    all_players_cleaned = [df for df in all_players if not df.empty]
    
    if all(all_players_cleaned[0].columns.tolist() == df.columns.tolist() for df in all_players_cleaned):
        all_players_df = pd.concat(all_players_cleaned, ignore_index=True)
    else:
        print("Nie można połączyć DataFrame'ów: różne kolumny lub typy danych.")
        return pd.DataFrame()

    all_player_attributes = []
    for _, player in all_players_df.iterrows():
        player_id = player['id']
        try:
            player_id = int(player_id)
        except ValueError:
            print(f"Błąd: ID zawodnika '{player_id}' nie jest liczbą całkowitą.")
            continue
        
        player_attributes = get_player_attributes_from_2022(player_id)
        if player_attributes:
            player_attributes['player_id'] = player_id
            player_attributes['team_id'] = player['team_id']
            player_attributes['team_name'] = player['team_name']
            player_attributes['league_id'] = player['league_id']
            player_attributes['league_name'] = player['league_name']
            all_player_attributes.append(player_attributes)
        time.sleep(1)
    
    all_player_attributes_df = pd.DataFrame(all_player_attributes)
    return all_player_attributes_df

def save_to_csv(df, filename):
    df.to_csv(filename, index=False)
    print(f"Dane zostały zapisane do pliku {filename}")

def save_to_csv(df, filename):
    df.to_csv(filename, index=False)
    print(f"Dane zostały zapisane do pliku {filename}")


In [ ]:
data = collect_data_from_2022()
if not data.empty:
    save_to_csv(data, 'd:\\praca_inzynierska\\Pliki_projektu\\player_attributes.csv')

In [3]:
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Dane zostały zapisane do pliku {filename}")

### Download teams and team's players

In [19]:
headers = {
    'X-Auth-Token': my_api_token
}

# Funkcja pobierająca dane z API
def get_data_from_api(url):
    while True:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            wait_time = int(response.headers.get('Retry-After', 1))
            print(f"Limit żądań przekroczony. Czekanie {wait_time} sekund.")
            time.sleep(wait_time)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

# Funkcja pobierająca listę drużyn
def get_all_teams():
    url = 'https://api.football-data.org/v4/teams'
    data = get_data_from_api(url)
    if data:
        teams = data['teams']
        return pd.DataFrame(teams)
    return pd.DataFrame()

# Zapisz listę drużyn do pliku CSV
teams_df = get_all_teams()
if not teams_df.empty:
    teams_df.to_csv(r'd:\\praca_inzynierska\\Pliki_projektu\\teams.csv', index=False)
    print("Drużyny zostały zapisane do pliku teams.csv")
else:
    print("Nie udało się pobrać listy drużyn.")

Drużyny zostały zapisane do pliku teams.csv


In [11]:
# Funkcja pobierająca listę zawodników dla danej drużyny
def get_players_for_team(team_id):
    url = f'https://api.football-data.org/v2/teams/{team_id}'
    data = get_data_from_api(url)
    if data and 'squad' in data:
        players = data['squad']
        return pd.DataFrame(players)
    return pd.DataFrame()

# Zapisz listę zawodników do pliku CSV
all_players = []

for _, team in teams_df.iterrows():
    team_id = team['id']
    players_df = get_players_for_team(team_id)
    if not players_df.empty:
        players_df['team_id'] = team_id
        players_df['team_name'] = team['name']
        all_players.append(players_df)
    else:
        print(f"Nie udało się pobrać zawodników z drużyny {team['name']}.")
    time.sleep(1)

if all_players:
    all_players_df = pd.concat(all_players, ignore_index=True)
    all_players_df.to_csv(r'd:\\praca_inzynierska\\Pliki_projektu\\players.csv', index=False)
    print("Zawodnicy zostali zapisani do pliku players.csv")
else:
    print("Nie udało się pobrać listy zawodników.")

Nie udało się pobrać zawodników z drużyny Arsenal FC.
Nie udało się pobrać zawodników z drużyny Aston Villa FC.
Nie udało się pobrać zawodników z drużyny Chelsea FC.
Nie udało się pobrać zawodników z drużyny Everton FC.
Nie udało się pobrać zawodników z drużyny Fulham FC.
Nie udało się pobrać zawodników z drużyny Liverpool FC.
Nie udało się pobrać zawodników z drużyny Manchester City FC.
Nie udało się pobrać zawodników z drużyny Manchester United FC.
Nie udało się pobrać zawodników z drużyny Newcastle United FC.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekanie 1 sekund.
Limit żądań przekroczony. Czekan

C:\Users\tysia\AppData\Local\Temp\ipykernel_22440\1911514644.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_players_df = pd.concat(all_players, ignore_index=True)
